<a href="https://colab.research.google.com/github/4GeeksAcademy/Proyecto_Final_Close-Loop-Intelligence/blob/main/src/Paso6%3A%20Construye_el_modelo_y_optim%C3%ADzalo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paso 6: Construye el modelo y optimízalo

### Elección del Modelo y Entrenamiento: Dada la naturaleza de clasificación, hemos decidido explorar los modelos de Regresión Logística, Random Forest y XGBoost

In [44]:
import pandas as pd

# train_data = pd.read_csv("/workspaces/Proyecto_Final_Close-Loop-Intelligence/data/processed/X_train_selected.csv")
# test_data = pd.read_csv("/workspaces/Proyecto_Final_Close-Loop-Intelligence/data/processed/X_test_selected.csv")
train_data = pd.read_csv("X_train_selected.csv")
test_data = pd.read_csv("X_test_selected.csv")

train_data.head()

,Ciudad Cliente,Ciudad Vendedor,Categoria Producto,Precio Unitario,Cantidad,Costo Flete,Total Orden,Estado Orden,Calificacion,Tiempo de Reposicion,ClasificacionABC
0,2,8,10,1415,0,2514,4569,0,3,9,0
1,1,206,26,224,0,1704,3312,0,0,21,1
2,10,348,37,3373,0,4605,27427,0,0,28,0
3,1,1,0,731,0,339,16135,0,0,0,0
4,2,481,11,466,0,45,9813,0,0,10,0


In [45]:
# Dividimos los datos en características (X) y variable objetivo (y)

X_train = train_data.drop(columns=['ClasificacionABC'])
y_train = train_data['ClasificacionABC']
X_test = test_data.drop(columns=['ClasificacionABC'])
y_test = test_data['ClasificacionABC']

In [46]:
# 1. Ver qué hay realmente en la columna
print(train_data['ClasificacionABC'].unique())

[0 1 2]


### 1) Modelo de Regresión Logística

In [47]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [48]:
# Predicciones y evaluación del modelo

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.82
Classification Report:
              precision    recall  f1-score   support

           0       0.82      1.00      0.90     16421
           1       0.00      0.00      0.00      2718
           2       0.00      0.00      0.00       897

    accuracy                           0.82     20036
   macro avg       0.27      0.33      0.30     20036
weighted avg       0.67      0.82      0.74     20036

Confusion Matrix:
[[16421     0     0]
 [ 2718     0     0]
 [  897     0     0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


El modelo predice solo la clase 0, entendemos que se debe al desbalance de clases.

In [49]:
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced')

In [50]:
# Predicciones y evaluación del modelo

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.36
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.36      0.51     16421
           1       0.13      0.33      0.19      2718
           2       0.05      0.36      0.09       897

    accuracy                           0.36     20036
   macro avg       0.34      0.35      0.26     20036
weighted avg       0.70      0.36      0.45     20036

Confusion Matrix:
[[5987 5609 4825]
 [ 906  886  926]
 [ 271  302  324]]


El accuracy ahora refleja la realidad, pero al menos el modelo es capaz de predecir cerca de un 30% en las clses B y C

### 2) Random Forest

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf_model = RandomForestClassifier(random_state = 42,class_weight='balanced')
rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [52]:
# Predicciones y evaluación del modelo

y_pred = rf_model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.84
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.97      0.91     16421
           1       0.65      0.24      0.35      2718
           2       0.60      0.21      0.31       897

    accuracy                           0.84     20036
   macro avg       0.70      0.47      0.52     20036
weighted avg       0.82      0.84      0.81     20036

Confusion Matrix:
[[15988   325   108]
 [ 2047   652    19]
 [  691    19   187]]


In [53]:

hyperparameters = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20, None],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Optimización del modelo

grid_search = GridSearchCV (rf_model, param_grid=hyperparameters, cv=3, scoring='f1_weighted', n_jobs=-1)
grid_search



GridSearchCV(cv=3,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, None],
                         'min_samples_leaf': [1, 2, 4],
                         'n_estimators': [50, 100]},
             scoring='f1_weighted')

In [55]:
# 1. Ajustar el modelo con los datos
grid_search.fit(X_train, y_train)

# 2. Obtener el mejor modelo
best_rf = grid_search.best_estimator_
print(f"Mejores parámetros: {grid_search.best_params_}")


Mejores parámetros: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 2, 'n_estimators': 100}


In [54]:
# 3. Predicciones y Evaluación
y_pred_rf = best_rf.predict(X_test)

print("\n--- Reporte de Clasificación (Random Forest Optimizado) ---")
report_rf_optimized = classification_report(y_test, y_pred_rf)
accuracy_rf_optimized = accuracy_score(y_test, y_pred_rf)
print(report_rf_optimized)

print("\n--- Matriz de Confusión ---")
print(confusion_matrix(y_test, y_pred_rf))


--- Reporte de Clasificación (Random Forest Optimizado) ---
              precision    recall  f1-score   support

           0       0.87      0.95      0.90     16421
           1       0.56      0.32      0.41      2718
           2       0.48      0.29      0.36       897

    accuracy                           0.83     20036
   macro avg       0.63      0.52      0.56     20036
weighted avg       0.81      0.83      0.81     20036


--- Matriz de Confusión ---
[[15525   651   245]
 [ 1801   879    38]
 [  603    37   257]]


### 3) XGBoost

In [56]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [57]:
# Predicciones y evaluación del modelo

y_pred = xgb_model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.83
Classification Report:
              precision    recall  f1-score   support

           0       0.83      1.00      0.91     16421
           1       0.79      0.09      0.17      2718
           2       0.90      0.10      0.17       897

    accuracy                           0.83     20036
   macro avg       0.84      0.40      0.42     20036
weighted avg       0.83      0.83      0.77     20036

Confusion Matrix:
[[16349    64     8]
 [ 2460   256     2]
 [  806     5    86]]


In [58]:
from sklearn.model_selection import RandomizedSearchCV

# Optimización del modelo

hyperparameters = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

random_search_xgb = RandomizedSearchCV(xgb_model, hyperparameters, n_iter=10, cv=3, scoring='f1_weighted', n_jobs=-1)

# 1. Ajustar el modelo con los datos
random_search_xgb.fit(X_train, y_train)

# 2. Obtener el mejor modelo
best_xgb = random_search_xgb.best_estimator_
print(f"Mejores parámetros: {random_search_xgb.best_params_}")

# 3. Predicciones y Evaluación
y_pred_xgb_optimized = best_xgb.predict(X_test)

print("--- Reporte de Clasificación (XGBoost Optimizado) ---")
report_xgb_optimized = classification_report(y_test, y_pred_xgb_optimized)
accuracy_xgb_optimized = accuracy_score(y_test, y_pred_xgb_optimized)
print(report_xgb_optimized)

print("")
print("--- Matriz de Confusión ---")
print(confusion_matrix(y_test, y_pred_xgb_optimized))


Mejores parámetros: {'subsample': 1.0, 'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.1, 'colsample_bytree': 0.8}
--- Reporte de Clasificación (XGBoost Optimizado) ---
              precision    recall  f1-score   support

           0       0.83      1.00      0.91     16421
           1       0.84      0.09      0.16      2718
           2       0.88      0.08      0.15       897

    accuracy                           0.83     20036
   macro avg       0.85      0.39      0.41     20036
weighted avg       0.84      0.83      0.77     20036


--- Matriz de Confusión ---
[[16366    45    10]
 [ 2475   243     0]
 [  823     1    73]]


### Conclusiones de la Comparativa de Modelos

Después de optimizar y evaluar los modelos de Random Forest y XGBoost, podemos extraer las siguientes conclusiones:

**1. Random Forest Optimizado:**
* **Accuracy:** 0.83 (83%)
* **Reporte de Clasificación:**
```
              precision    recall  f1-score   support

           0       0.87      0.95      0.90     16421
           1       0.56      0.32      0.41      2718
           2       0.48      0.29      0.36       897

    accuracy                           0.83     20036
   macro avg       0.63      0.52      0.56     20036
weighted avg       0.81      0.83      0.81     20036
```
* **Análisis:** El modelo Random Forest, tras la optimización con `GridSearchCV` y el uso de `class_weight='balanced'`, logró una mejora significativa en la detección de las clases minoritarias (1 y 2). Aunque la precisión general (accuracy) es de 0.83, es importante destacar que el recall para la clase 1 es del 32% y para la clase 2 es del 29%. Esto indica que, aunque todavía hay margen de mejora en la identificación de estas clases, el modelo no se limita a predecir solo la clase mayoritaria.

**2. XGBoost Optimizado:**
* **Accuracy:** 0.83 (83%)
* **Reporte de Clasificación:**
```
              precision    recall  f1-score   support

           0       0.83      1.00      0.91     16421
           1       0.84      0.09      0.16      2718
           2       0.88      0.08      0.15       897

    accuracy                           0.83     20036
   macro avg       0.85      0.39      0.41     20036
weighted avg       0.84      0.83      0.77     20036
```
* **Análisis:** El modelo XGBoost, optimizado con `RandomizedSearchCV`, también alcanza un accuracy del 0.83. Sin embargo, al observar el reporte de clasificación, se evidencia que este modelo tiene dificultades para predecir las clases minoritarias (1 y 2), con recalls de solo 9% y 8% respectivamente. Esto sugiere que, a pesar de su alta precisión para la clase 0, el modelo tiende a clasificar la mayoría de las instancias como la clase predominante.

**Conclusión General:**

Ambos modelos lograron un accuracy similar. Sin embargo, considerando la naturaleza del problema de clasificación (con un desbalance de clases significativo), el **Random Forest optimizado** demuestra ser un modelo más equilibrado. Aunque su recall en las clases minoritarias no es extremadamente alto, es sustancialmente mejor que el de XGBoost, lo que lo hace más útil para identificar las instancias de las clases 1 y 2, que probablemente son de mayor interés para el negocio.


In [62]:
# Guardado del modelo

import joblib

joblib.dump(best_rf, 'best_rf_model.pkl')


['best_rf_model.pkl']